In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import numpy.linalg as la

# Dimension Reduction

In [ ]:
np.random.seed(123)
np.set_printoptions(3)

### PCA from scratch

Principal Components Analysis (PCA) basically means to find and rank all the eigenvalues and eigenvectors of a covariance matrix. This is useful because high-dimensional data (with $p$ features) may have nearly all their variation in a small number of dimensions $k$, i.e. in the subspace spanned by the eigenvectors of the covariance matrix that have the $k$ largest eigenvalues. If we project the original data into this subspace, we can have a dimension reduction (from $p$ to $k$) with hopefully little loss of information.

For zero-centered vectors,

\begin{align}
\text{Cov}(X, Y) &= \frac{\sum_{i=1}^n(X_i - \bar{X})(Y_i - \bar{Y})}{n-1} \\
  &= \frac{\sum_{i=1}^nX_iY_i}{n-1} \\
  &= \frac{XY^T}{n-1}
\end{align}

and so the covariance matrix for a data set X that has zero mean in each feature vector is just $XX^T/(n-1)$. 

In other words, we can also get the eigendecomposition of the covariance matrix from the positive semi-definite matrix $XX^T$.

We will take advantage of this when we cover the SVD later in the course.

Note: Here we are using a matrix of **row** vectors

In [ ]:
n = 100
x1, x2, x3 = np.random.normal(0, 10, (3, n))
x4 = x1 + np.random.normal(size=x1.shape)
x5 = (x1 + x2)/2 + np.random.normal(size=x1.shape)
x6 = (x1 + x2 + x3)/3 + np.random.normal(size=x1.shape)

#### For PCA calculations, each column is an observation

In [ ]:
xs = np.c_[x1, x2, x3, x4, x5, x6].T

In [ ]:
xs[:, :10]

#### Center each observation

In [ ]:
xc = xs - np.mean(xs, 1)[:, np.newaxis]

In [ ]:
xc[:, :10]

#### Covariance

Remember the formula for covariance

$$
\text{Cov}(X, Y) = \frac{\sum_{i=1}^n(X_i - \bar{X})(Y_i - \bar{Y})}{n-1}
$$

where $\text{Cov}(X, X)$ is the sample variance of $X$.

In [ ]:
cov = (xc @ xc.T)/(n-1)

In [ ]:
cov

#### Check

In [ ]:
np.cov(xs)

#### Eigendecomposition

In [ ]:
e, v = la.eigh(cov)

In [ ]:
idx = np.argsort(e)[::-1]

In [ ]:
e = e[idx]
v = v[:, idx]

#### Explain the magnitude of the eigenvalues

Note that $x4, x5, x6$ are linear combinations of $x1, x2, x3$ with some added noise, and hence the last 3 eigenvalues are small. 

In [ ]:
plt.stem(e)
pass

#### The eigenvalues and eigenvectors give a factorization of the covariance matrix

In [ ]:
v @ np.diag(e) @ v.T

### Geometry of PCA

![Geometry off PCA](https://i.stack.imgur.com/AaF1w.jpg)

### Algebra of PCA

![Commuative diagram](figs/spectral.png)

Note that $Q^T X$ results in a new data set that is uncorrelated.

In [ ]:
m = np.zeros(2)
s = np.array([[1, 0.8], [0.8, 1]])
x = np.random.multivariate_normal(m, s, n).T

In [ ]:
x.shape

#### Calculate covariance matrix from centered observations

In [ ]:
xc = (x - x.mean(1)[:, np.newaxis])

In [ ]:
cov = (xc @ xc.T)/(n-1)

In [ ]:
cov

#### Find eigendecoposition

In [ ]:
e, v = la.eigh(cov)
idx = np.argsort(e)[::-1]
e = e[idx]
v = v[:, idx]

#### In original coordinates

In [ ]:
plt.scatter(x[0], x[1], alpha=0.5)
for e_, v_ in zip(e, v.T):
    plt.plot([0, e_*v_[0]], [0, e_*v_[1]], 'r-', lw=2)
plt.xlabel('x', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.axis('square')
plt.axis([-3,3,-3,3])
pass

#### After change of basis

In [ ]:
yc = v.T @ xc

In [ ]:
plt.scatter(yc[0,:], yc[1,:], alpha=0.5)
for e_, v_ in zip(e, np.eye(2)):
    plt.plot([0, e_*v_[0]], [0, e_*v_[1]], 'r-', lw=2)
plt.xlabel('PC1', fontsize=14)
plt.ylabel('PC2', fontsize=14)
plt.axis('square')
plt.axis([-3,3,-3,3])
pass

#### Check

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

#### Note that the PCA from scikit-learn works with feature vectors, not observation vectors

In [ ]:
z = pca.fit_transform(x.T)

#### Eigenvectors from PCA

In [ ]:
pca.components_

#### Eigenvalues from PCA

In [ ]:
pca.explained_variance_

#### Explained variance

This is just a consequence of the invariance of the trace under change of basis. Since the original diagnonal entries in the covariance matrix are the variances of the featrues, the sum of the eigenvalues must also be the sum of the orignal varainces. In other words, the cumulateive proportion of the top $n$ eigenvaluee is the "explained variance" of the first $n$ principal components. 

In [ ]:
e/e.sum()

In [ ]:
pca.explained_variance_ratio_

#### The principal components are identical to our home-brew version, up to a flip in direction of eigenvectors

In [ ]:
plt.scatter(z[:, 0], z[:, 1], alpha=0.5)
for e_, v_ in zip(e, np.eye(2)):
    plt.plot([0, e_*v_[0]], [0, e_*v_[1]], 'r-', lw=2)
plt.xlabel('PC1', fontsize=14)
plt.ylabel('PC2', fontsize=14)
plt.axis('square')
plt.axis([-3,3,-3,3])
pass

In [ ]:
plt.subplot(121)
plt.scatter(-z[:, 0], -z[:, 1], alpha=0.5)
for e_, v_ in zip(e, np.eye(2)):
    plt.plot([0, e_*v_[0]], [0, e_*v_[1]], 'r-', lw=2)
plt.xlabel('PC1', fontsize=14)
plt.ylabel('PC2', fontsize=14)
plt.axis('square')
plt.axis([-3,3,-3,3])
plt.title('Scikit-learn PCA (flipped)')

plt.subplot(122)
plt.scatter(yc[0,:], yc[1,:], alpha=0.5)
for e_, v_ in zip(e, np.eye(2)):
    plt.plot([0, e_*v_[0]], [0, e_*v_[1]], 'r-', lw=2)
plt.xlabel('PC1', fontsize=14)
plt.ylabel('PC2', fontsize=14)
plt.axis('square')
plt.axis([-3,3,-3,3])
plt.title('Homebrew PCA')
plt.tight_layout()
pass